# JSON-tiedostot Parquet-muotoon
- Luetaan JSON-tiedostot hakemistosta
- Poistetaan tarpeettomat sarakkeet
- Normalisoidaan JSON
- Tallennetaan jokainen omaksi parquet-tiedostokseen

In [13]:
import pandas as pd
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import datetime as dt
import os


file_path = r'Data\Temp'
output_dir = r'Data\Processed'
output_file = r'Data\vr_trains_2026_01.parquet'

folders = [f for f in os.listdir(file_path) if os.path.isdir(os.path.join(file_path, f))]


def create_modified_df(df):
    print(f'df shape: {df.shape}')
    columns = ['departureDate', 'trainNumber', 'trainCategory', 'trainType', 'commuterLineID', 'stationShortCode', 'type', 
           'trainStopping', 'commercialStop', 'scheduledTime', 'actualTime', 'differenceInMinutes', 'operatorUICCode', 'operatorShortCode', 'cancelled']
    
    df = df.drop(columns=['runningCurrently', 'version', 'timetableType', 'timetableAcceptanceDate', 'cancelled'], errors='ignore')
    trains = df.explode('timeTableRows', ignore_index=True)
    print(f'df shape exploden jälkeen: {trains.shape}')
    trains = trains.drop(columns=['cancelled'], errors='ignore')
    tt_rows_df = pd.json_normalize(trains['timeTableRows'])[['stationShortCode', 'trainStopping', 'commercialStop', 'type', 'scheduledTime',
                                                             'actualTime', 'differenceInMinutes', 'cancelled']]
    
    trains = trains.drop(columns=['timeTableRows'], errors='ignore')
    trains[tt_rows_df.columns] = tt_rows_df

    trains = trains.reindex(columns=columns)
    print(f'df shape yhdistämisen ja turhien sarakkeiden poiston jälkeen: {trains.shape}')

    return trains



for i, file in enumerate(os.listdir(file_path)):
    print(f'Käsitellään tiedostoa: {file}')
    temp_df = pd.read_json(os.path.join(file_path, file))
    trains = create_modified_df(temp_df) 

    out = os.path.join(output_dir, f'file{i}.parquet')

    print(f'Luodaan parquet-tiedosto)...')
    trains.to_parquet(
        out,
        index=False,
        engine='pyarrow',
        compression='snappy',
    )

print('Solun suoritus valmis.')

Käsitellään tiedostoa: 2026-01-01_trains.json
df shape: (1423, 13)
df shape exploden jälkeen: (62744, 8)
df shape yhdistämisen ja turhien sarakkeiden poiston jälkeen: (62744, 15)
Luodaan parquet-tiedosto)...
Käsitellään tiedostoa: 2026-01-02_trains.json
df shape: (1734, 13)
df shape exploden jälkeen: (73070, 8)
df shape yhdistämisen ja turhien sarakkeiden poiston jälkeen: (73070, 15)
Luodaan parquet-tiedosto)...
Käsitellään tiedostoa: 2026-01-03_trains.json
df shape: (1531, 13)
df shape exploden jälkeen: (64492, 8)
df shape yhdistämisen ja turhien sarakkeiden poiston jälkeen: (64492, 15)
Luodaan parquet-tiedosto)...
Käsitellään tiedostoa: 2026-01-04_trains.json
df shape: (1362, 13)
df shape exploden jälkeen: (61124, 8)
df shape yhdistämisen ja turhien sarakkeiden poiston jälkeen: (61124, 15)
Luodaan parquet-tiedosto)...
Käsitellään tiedostoa: 2026-01-05_trains.json
df shape: (1793, 13)
df shape exploden jälkeen: (73274, 8)
df shape yhdistämisen ja turhien sarakkeiden poiston jälkeen: (

## Yhdistetään 365 parquet-tiedostoa yhdeksi tiedostoksi.

In [14]:
dataset = ds.dataset(output_dir, format="parquet")
table = dataset.to_table()
pq.write_table(table, output_file, compression='snappy')

print(f'trains df sisältää {table.num_rows} riviä ja {table.num_columns} saraketta.')

trains df sisältää 2178810 riviä ja 15 saraketta.


In [15]:
df = pd.read_parquet('Data/vr_trains_2026_01.parquet')
print(df['departureDate'].min(), df['departureDate'].max())

2026-01-01 2026-01-31
